In [107]:
library(repr)    # for figure size
library(randomForest)

In [108]:
is.Date <- function(x) inherits(x, 'Date')

In [109]:
RF.res <- readRDS("model-RF-res.rds")

# Importation du test modifié

In [133]:
test <- read.csv('test_modifié.csv', row.names=1)

## Remise au format date

In [130]:
test$YearBuilt <- as.Date(
    ISOdate(test[, 'YearBuilt'], 01, 01), format = "Y-%m-%d")
test$YearRemodAdd <- as.Date(
    ISOdate(test[, 'YearRemodAdd'], 01, 01), format = "Y-%m-%d")
test$GarageYrBlt <- as.Date(
    ISOdate(test[, 'GarageYrBlt'], 01, 01), format = "Y-%m-%d")
test$YrSold <- as.Date(test[, 'YrSold'], format = "%Y-%m-%d")

In [134]:
options(repr.matrix.max.cols=200)
head(test)

,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterCond,Foundation,BsmtQual,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtUnfSF,Heating,HeatingQC,CentralAir,Electrical,X1stFlrSF,X2ndFlrSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,PoolQC,Fence,MiscFeature,YrSold,SaleType,SaleCondition
,<int>,<fct>,<int>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>
1461,20,RH,80,11622,None,4,Inside,NAmes,Feedr,1Fam,1Story,6,1961-01-01,1961-01-01,Gable,0,VinylSd,VinylSd,None,0,3,CBlock,3,1,3,468,270,0,3,1,5,896,0,0,0,1,0,2,3,5,0,None,Attchd,1961-01-01,Unf,730,3,140,0,None,MnPrv,0,2010-06-01,WD,Normal
1462,20,RL,81,14267,None,3,Corner,NAmes,Norm,1Fam,1Story,6,1958-01-01,1958-01-01,Hip,0,Wd Sdng,Wd Sdng,BrkFace,108,3,CBlock,3,1,5,923,406,0,3,1,5,1329,0,0,0,1,1,3,4,6,0,None,Attchd,1958-01-01,Unf,312,3,393,36,None,None,1,2010-06-01,WD,Normal
1463,60,RL,74,13830,None,3,Inside,Gilbert,Norm,1Fam,2Story,5,1997-01-01,1998-01-01,Gable,0,VinylSd,VinylSd,None,0,3,PConc,4,1,6,791,137,0,4,1,5,928,701,0,0,2,1,3,3,6,1,TA,Attchd,1997-01-01,Fin,482,3,212,34,None,MnPrv,0,2010-03-01,WD,Normal
1464,60,RL,78,9978,None,3,Inside,Gilbert,Norm,1Fam,2Story,6,1998-01-01,1998-01-01,Gable,0,VinylSd,VinylSd,BrkFace,20,3,PConc,3,1,6,602,324,0,5,1,5,926,678,0,0,2,1,3,4,7,1,Gd,Attchd,1998-01-01,Fin,470,3,360,36,None,None,0,2010-06-01,WD,Normal
1465,120,RL,43,5005,None,3,Inside,StoneBr,Norm,TwnhsE,1Story,5,1992-01-01,1992-01-01,Gable,0,HdBoard,HdBoard,None,0,3,PConc,4,1,5,263,1017,0,5,1,5,1280,0,0,0,2,0,2,4,5,0,None,Attchd,1992-01-01,RFn,506,3,0,82,None,None,0,2010-01-01,WD,Normal
1466,60,RL,75,10000,None,3,Corner,Gilbert,Norm,1Fam,2Story,5,1993-01-01,1994-01-01,Gable,0,HdBoard,HdBoard,None,0,3,PConc,4,1,1,0,763,0,4,1,5,763,892,0,0,2,1,3,3,7,1,TA,Attchd,1993-01-01,Fin,440,3,157,84,None,None,0,2010-04-01,WD,Normal


In [113]:
dim(test)

[1] 1152   55

## Complément des modalités manquantes dans le set
- => doit avoir même modalités que le setde train utilisé pour entrâiner le modèle

In [114]:
levels(test$Exterior1st) <- c(levels(test$Exterior1st), 'AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing')
levels(test$Exterior2nd) <- c(levels(test$Exterior2nd), 'AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing')
levels(test$Foundation) <- c(levels(test$Foundation), 'BrkTil', 'CBlock', 'PConc', 'Slab', 'Stone', 'Wood')
levels(test$GarageType) <- c(levels(test$GarageType), '2Types', 'Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd')
levels(test$LotConfig) <- c(levels(test$LotConfig), 'Inside', 'Corner', 'CulDSac', 'FR2', 'FR3')
levels(test$MasVnrType) <- c(levels(test$MasVnrType), 'BrkCmn', 'BrkFace', 'CBlock', 'None', 'Stone')
levels(test$MSZoning) <- c(levels(test$MSZoning), 'A', 'C', 'FV', 'I', 'RH', 'RL', 'RP', 'RM')
levels(test$PoolQC) <- c(levels(test$PoolQC), 'Po', 'Fa', 'TA', 'Gd', 'Ex')
levels(test$RoofStyle) <- c(levels(test$RoofStyle), 'Flat', 'Gable', 'Gambrel', 'Hip', 'Mansard', 'Shed')
levels(test$SaleCondition) <- c(levels(test$SaleCondition), 'Normal', 'Abnorml', 'AdjLand', 'Alloca', 'Family', 'Partial')
levels(test$SaleType) <- c(levels(test$SaleType), 'WD ', 'CWD', 'VWD', 'New', 'COD', 'Con', 'ConLw', 'ConLI', 'ConLD', 'Oth')
levels(test$HouseStyle) <- c(levels(test$HouseStyle), '1Story', '1.5Fin', '1.5Unf', '2Story', '2.5Fin', '2.5Unf', 'SFoyer', 'SLvl')

# Génération des prédictions sur test et mise au format

In [115]:
RF.test.preds <- predict(RF.res, test)

In [142]:
class(test)

[1] "data.frame"

In [118]:
class(RF.test.preds)

[1] "numeric"

In [145]:
RF.test.preds.df <- as.data.frame(RF.test.preds)

In [146]:
RF.test.preds.df <- data.frame(row.names(RF.test.preds.df), RF.test.preds.df)

In [147]:
names(RF.test.preds.df) <- c('Id', 'SalePrice')
head(RF.test.preds.df)

,Id,SalePrice
,<fct>,<dbl>
1461,1461,131863.3
1462,1462,156554.8
1463,1463,191078.4
1464,1464,194501.6
1465,1465,190959.3
1466,1466,196478.7


In [150]:
which(is.na(RF.test.preds.df))

[1] 1237 1709 1974 2042

# Exportation du résultat avec les prédictions

In [148]:
write.csv(RF.test.preds.df, 'test_predict.csv', row.names=F)